## Satellite Data

From MAPBIOMAS:
- secondary forest age
- last observed land use type
- number of years under each land use type
- number of fires
- time since last fire
- fallow period length

From ESA CCI Biomass
- biomass (Mg C/hectare)

From Ma et al 2023
- fragmentation for 2020

In [26]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize(project='ee-ana-zonia')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

first_year = 1985
last_year = 2020

from utils import export_image
from utils import map_image

In [11]:
# import ages from MapBiomas
age = ee.Image('projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_secondary_vegetation_age_v1').select('secondary_vegetation_age_2020')
# region of interest is the entire country
roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()

# biomass for 2020 comes from CCI Biomass
biomass = ee.Image("projects/ee-ana-zonia/assets/biomass")
proj = biomass.projection().getInfo()
crs = proj['crs']
crsTransform = proj['transform']

In [18]:
basemap = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            .map(lambda image: image.clip(roi))

vis = {'bands':['R','G','B'],'min':64,'max':5454,'gamma':1.8}

In [32]:

kernel = ee.Kernel.square(radius = 100, units = 'meters')

age_zeroes = age.unmask(0)
age_mask = age_zeroes.reduceNeighborhood(reducer = ee.Reducer.mode(), kernel = kernel)
age_mask = age_mask.updateMask(age_mask)

# export_image(age_mask, 'one_hectare_secondary_forest_mask')
# Map = geemap.Map()
# Map.addLayer(age_mask, {}, 'age_mask')
# Map.addLayer(age, {}, 'age')
# Map

# make 1/0 distinction between isolated pixels and 100 m forests

In [16]:
# Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
lulc = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1") \
  .select([f"classification_{year}" for year in range(first_year, last_year+1)]).byte()

lulc_masked = lulc.updateMask(age.mask())

In [313]:
# Remove secondary forests with undesired histories
oldvalues = ee.List([3, 6, 15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21])
newvalues = ee.List.repeat(1, oldvalues.size())

# Get the band names
band_names = lulc_masked.bandNames()


In [314]:

# Define a function to apply remap to a band
def remap_band(band_name):
    band = lulc_masked.select(ee.String(band_name))
    new_band = band.remap(oldvalues, newvalues, 0)
    return new_band.rename(ee.String(band_name))

# Map the function over the band names
remapped_image = band_names.map(remap_band)
remapped_image = ee.ImageCollection(remapped_image).toBands()
desired_mask = remapped_image.reduce('sum').eq(36)

lulc_masked = lulc_masked.updateMask(desired_mask)

In [315]:
age = age.updateMask(desired_mask)

fire = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-annual-burned-coverage-1") \
  .select([f"burned_coverage_{year}" for year in range(first_year, last_year)]).byte()
fire = fire.updateMask(age.mask())

### Fire
Note that fire has different transform than lulc, so the projection will need to be adjusted when exporting.

In [283]:
# fire has the value of the land use type that burned.
# Transforming into a fire mask:
fire = fire.gt(0)
num_fires = fire.reduce(ee.Reducer.sum()).rename('num_fires').byte()

In [291]:
# how many years ago was EACH fire? #############################
# Get the number of bands
num_bands = fire.bandNames().size()
# Create a sequence of numbers from 1 to num_bands
years_ago = ee.List.sequence(1, num_bands)
years_ago = years_ago.reverse()

# Map over the image and set values based on the band index
constant_images = ee.ImageCollection.fromImages(
    years_ago.map(lambda year: ee.Image.constant(year))).toBands()

time_since_all_fires = fire.multiply(constant_images)

old_names = time_since_all_fires.bandNames().getInfo()
new_names = [name.replace('burned_coverage', 'time_since_fire') for name in old_names]
time_since_all_fires = time_since_all_fires.select(old_names).rename(new_names)

In [300]:
# how many years ago was the LAST fire? #############################
last_fire = time_since_all_fires.reduce(ee.Reducer.lastNonNull()).rename('last_fire').byte()

In [295]:
# get fire frequency data from Mapbiomas - double check it.
fire_freq = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-fire-frequency-1").clip(roi)
fire_freq = fire_freq.select('fire_frequency_1985_2020').clip(ecoregions.first())

### Total sum of land use types

In [316]:
LU_index = [15, 39, 20, 40, 62, 41, 46, 47, 35, 48, 9, 21]

LU_sum = ee.Image()

for val in LU_index:
  lulc_val = lulc_masked.eq(val)
  num_cells = lulc_val.reduce(ee.Reducer.sum()).rename(f'lulc_sum_{val}')
  LU_sum = LU_sum.addBands(num_cells)

LU_sum = LU_sum.slice(1).byte()

### Last Land Use Type

In [317]:
years = range(1986, 2020)

last_LU = ee.Image()

for yr in years:
    year = f'classification_{yr}'
    lu_yr = lulc_masked.select(year)
    age_mask = age.eq(2020 - yr)
    last_LU = last_LU.addBands(lu_yr.updateMask(age_mask).updateMask(lu_yr.neq(3)).updateMask(lu_yr.neq(6)))

last_LU = last_LU.slice(1)

# Convert ImageCollection to single Image
last_LU = last_LU.reduce(ee.Reducer.sum()).rename('last_LU')

### Fallow period length

In [330]:
nat_cover = lulc_masked.updateMask((lulc.neq(1).neq(4)).neq(5)).lt(7)
total_nat_cover = nat_cover.reduce(ee.Reducer.sum())
fallow = total_nat_cover.subtract(age).rename('fallow')

In [347]:
# export_image(last_LU, "last_LU")
# export_image(LU_sum, "LU_sum")
# export_image(last_fire, "last_fire")
# export_image(num_fires, "num_fires")
# export_image(age, "age")
# export_image(fallow, "fallow")
# export_image(lulc_masked, "lulc")
# export_image(biomass, "biomass_masked")

In [339]:
# Reproject to 10m
biomass = biomass.reproject(crs=age.projection(), scale=10)
# Reaggregate to 30m (mean value)
biomass = biomass.reduceResolution(reducer=ee.Reducer.mean()).reproject(crs=age.projection())
# Mask only to regions with age greater than zero (secondary forests)
biomass = biomass.updateMask(age.mask()).int16().rename('agbd')

In [365]:
# Mask the image to keep only pixels with natural vegetation cover in all bands
# 3 = forest
# 6 = flooded forest
biomass = ee.Image("projects/ee-ana-zonia/assets/biomass")

mature_cover = lulc.eq(3).Or(lulc.eq(6))
mature_mask = mature_cover.reduce(ee.Reducer.allNonZero())
mature_biomass = biomass.updateMask(mature_mask.mask())

vis = {
    'min': 0,
    'max': 400,
    'palette': ['blue', 'red'],
}

Map = geemap.Map()
Map.addLayer(mature_mask, {}, 'mature_mask')
Map.addLayer(lulc_masked, {}, 'lulc_masked')
Map.addLayer(mature_biomass, vis, 'mature_biomass')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:

# Compute the mean biomass values for mature forest per ecoregion.
median_mature = mature_biomass.reduceRegions(ecoregions, reducer = ee.Reducer.median(),\
                                             scale = 10000, crs = crs)

# Convert the FeatureCollection to an image.
median_mature = median_mature.reduceToImage(['median'], ee.Reducer.first())



In [30]:
import geemap

# Load the NICFI image collection
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

# Filter basemaps by date
basemap = nicfi.filterDate('2020-01-01', '2020-12-31').first().clip(

# Center the map on the basemap
Map.centerObject(basemap, 4)

# Visualization parameters
vis = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}

# Add the basemap layer to the map
Map.addLayer(basemap, vis, '2020 Planet')

# Calculate and add the NDVI layer
ndvi = basemap.normalizedDifference(['N', 'R']).rename('NDVI')
ndvi_vis = {'min': -0.55, 'max': 0.8, 'palette': ['8bc4f9', 'c9995c', 'c7d270', '8add60', '097210']}
Map.addLayer(ndvi, ndvi_vis, 'NDVI', False)

# Display the map
Map


Map(bottom=1064417.0, center=[-0.03247460684437231, -73.82115720575285], controls=(WidgetControl(options=['pos…